# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
print(len(data))

4870


## Question 1

A permutation test is appropriate here. CLT does hold because there are 4870 observations and each observation is independent from every other observation.

## Question 2

Null Hypothesis: The distributions of black callbacks is identical to white callbacks. 
Alternative: The distributions of black call backs is not identical to white callbacks.

## Question 3

In [6]:
def permutation_sample(data1,data2):
    """Generate a permutation sample from two data sets."""
    data = np.concatenate((data1,data2))
    permuted_data = np.random.permutation(data)
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]
    
    return perm_sample_1,perm_sample_2

def draw_perm_reps(data_1,data_2,func,size=1):
    """Generate multiple permutation replicates."""
    perm_replicates = np.empty(size)
    
    for i in range(size):
        perm_sample_1,perm_sample_2 = permutation_sample(data_1,data_2)
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)
        
    return perm_replicates

In [10]:
#isolate callback data of black resumes
black = data.call[data.race == 'b']
black.head()

2    0.0
3    0.0
7    0.0
8    0.0
9    0.0
Name: call, dtype: float32

In [11]:
#isolate callback data of white resumes
white = data.call[data.race == 'w']
white.head()

0    0.0
1    0.0
4    0.0
5    0.0
6    0.0
Name: call, dtype: float32

In [14]:
#calculate the proportion of callbacks for black and white resumes
print((np.sum(black)/len(black),np.sum(white)/len(white)))

(0.064476386036960986, 0.096509240246406572)


In [17]:
def diff_proportion(data1,data2):
    data1_prop = np.sum(data1)/len(data1)
    data2_prop = np.sum(data2)/len(data2)
    
    return data1_prop - data2_prop

In [28]:
#create 10,000 permutations of the difference of proportions of white and black callbacks
replicates = draw_perm_reps(black,white,diff_proportion,10000)

In [29]:
#calculate observed difference in proportion of callbacks
observed_diff_proportion = diff_proportion(black,white)
observed_diff_proportion

-0.032032854209445585

In [30]:
#calculate p value
p_value = np.sum(replicates <= observed_diff_proportion) / len(replicates)
p_value

0.0

In [36]:
#calculate 95% confidence intervals
confidence_int = np.percentile(replicates,[2.5,97.5])
confidence_int

array([-0.01560575,  0.01560575])

In [ ]:
margin_of_error = .0156

## Question 4

I sought to determine whether race has a significant impact on the rate of callbacks. In order to test this, I assumed that the distribution of proportion of callbacks of black and white resumes where identical. I ran 10,000 replicates under this assumption. Of the 10,000 replicates, none of the differences in proportions of black and white callbacks were of larger magnitude than the observed difference in rate of callbacks. Therefore, there is sufficient evidence that the distribution of the proportion of callbacks of black and white resumes are not identical. 

## Question 5

My analysis does not mean that race is the most important factor in callback success. My analysis concludes that there is a significant difference of callback success based on race. In order to find the most important factor in callback success, I would calculate correlation between callback success and other variables available in the dataset.